In [1]:
import pandas as pd
import numpy as np 
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import requests
import re
from keybert import KeyBERT

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel("feedback student.xlsx")

In [3]:
data.head()

,Id,comment,quality
0,45881,"Rand was a good guy, really... he was. But I f...",awful
1,45881,"He is brilliant in his field, but expects his ...",poor
2,45881,He has a major in philosophy which is why all ...,awful
3,45881,I thought his class was extremely hard but des...,good
4,45881,I recently just finished taking Comp 110 with ...,poor


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2345 entries, 0 to 2344
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Id       2345 non-null   int64 
 1   comment  2345 non-null   object
 2   quality  2345 non-null   object
dtypes: int64(1), object(2)
memory usage: 55.1+ KB


In [5]:
data = data.drop(columns=['quality'])

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [7]:
# Testing the model
tokens = tokenizer.encode(data['comment'][100],return_tensors='pt')

In [8]:
result = model(tokens)

In [9]:
result.logits

tensor([[ 3.1769,  2.7265,  0.8143, -2.2313, -3.5469]],
       grad_fn=<AddmmBackward0>)

In [10]:
int(torch.argmax(result.logits))+1

1

In [11]:
# Defining a function for score
def sentiment_score(review):
    tokens = tokenizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits)) + 1

In [12]:
data['sentiment_score'] = data['comment'].apply(lambda x :sentiment_score(x[:512]))

In [13]:
data['sentiment_label'] = data['sentiment_score'].apply(
    lambda x: 'negative' if x < 3 else 'neutral' if x == 3 else 'positive'
)


In [14]:
data

,Id,comment,sentiment_score,sentiment_label
0,45881,"Rand was a good guy, really... he was. But I f...",3,neutral
1,45881,"He is brilliant in his field, but expects his ...",4,positive
2,45881,He has a major in philosophy which is why all ...,3,neutral
3,45881,I thought his class was extremely hard but des...,4,positive
4,45881,I recently just finished taking Comp 110 with ...,2,negative
...,...,...,...,...
2340,64144,"He's really disoriented, but a really cool guy...",4,positive
2341,64144,No Comments,1,negative
2342,64144,Horrible,1,negative
2343,64144,No Comments,1,negative


In [16]:
kw_model = KeyBERT()

In [27]:
def extract_key_phrases(text):
    if not isinstance(text, str) or text.strip() == "":
        return []
    
    keywords = kw_model.extract_keywords(
        text,
        keyphrase_ngram_range=(1,10),
        stop_words='english',
        top_n=3
    )
    return [kw[0] for kw in keywords]


In [28]:
data['key_phrases'] = data['comment'].apply(extract_key_phrases)

In [29]:
data.head()

,Id,comment,sentiment_score,sentiment_label,key_phrases
0,45881,"Rand was a good guy, really... he was. But I f...",3,neutral,[rand good guy really felt alter ego left home...
1,45881,"He is brilliant in his field, but expects his ...",4,positive,"[brilliant field expects students perform, bri..."
2,45881,He has a major in philosophy which is why all ...,3,neutral,[write based philosophy long paper agrees thin...
3,45881,I thought his class was extremely hard but des...,4,positive,"[teacher want learn rand eng 110 worth, differ..."
4,45881,I recently just finished taking Comp 110 with ...,2,negative,"[taking comp 110 rand hardest class, taking co..."


In [32]:
data.comment[100]

"The amount of homework is absolutely ridiculous and he doesn't explain anything very well in calss"

In [33]:
data.key_phrases[100]

['homework absolutely ridiculous doesn explain calss',
 'absolutely ridiculous doesn explain calss',
 'doesn explain calss']

In [34]:
data.to_csv("student_feedback_sentiment.csv", index=False)